# Шаг 1. Парсинг данных

### Парсинг данных индексов IMOEX, MOEXMM, MOEXOG ###

Код основан на MOEX API с использованием https://iss.moex.com/iss/reference/

In [42]:
# импорт библиотек для парсинга
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#для индекса IMOEX

def get_imoex(start_date, end_date, page_size = 100):

    '''Функция зависит от переменных: start_date - дата начала сбора данных, end_date - дата окончания сбора данных,
    page_size - количество данных на одной странице, равное 100. Внутри функции создается список imoex_data с хранением информации, счетчик
    count отсчитывает данные, находящиеся на одной странице и служит индикатором перехода на другую. '''

    imoex_data = []
    count = 0
    while True:
        # используя MOEX ISS получаем url сайта
        url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/IMOEX.json?from={start_date}&till={end_date}&start={count}&limit={page_size}'
        try:
            response = requests.get(url, timeout=15) # получаем доступ к сайту и устанавливаем максимальное время ответа
            data = response.json() # получаем данные из API и переводим из JSON в python формат
            history_data = data['history']['data'] # !!!!!!!!!!!
            headings = data['history']['columns']
            if history_data != None: 
                imoex_data.extend(history_data)
                count += len(history_data) # увеличиваем счётчик, для того чтобы на следующей странице начать с этого номера
        except:
            return pd.DataFrame(imoex_data, columns=headings) # если данные закончились возвращаем imoex_data
            
        if len(history_data) < page_size: # останавливаем бесконечный цикл
            break
    return pd.DataFrame(imoex_data, columns=headings)

# Устанавливаем даты старта и окончания сбора, так, чтобы они соответствовали дата-фрейму с сентимент анализом.
# При этом MOEX не публиковало рыночную капитализацию до 
start_date = '2012-12-01'
end_date = '2024-01-31'
page_size = 100
df = get_imoex(start_date, end_date, page_size=page_size)
print(df.head().to_string())

  BOARDID  SECID   TRADEDATE        SHORTNAME             NAME    CLOSE     OPEN     HIGH      LOW         VALUE  DURATION  YIELD  DECIMALS  CAPITALIZATION CURRENCYID  DIVISOR TRADINGSESSION VOLUME
0    SNDX  IMOEX  2012-12-03  Индекс МосБиржи  Индекс МосБиржи  1421.97  1405.97  1422.20  1403.70  2.559112e+10       NaN    NaN         2    3.927751e+12        RUB      NaN              3   None
1    SNDX  IMOEX  2012-12-04  Индекс МосБиржи  Индекс МосБиржи  1411.83  1421.94  1421.94  1407.84  2.296478e+10       NaN    NaN         2    3.899745e+12        RUB      NaN              3   None
2    SNDX  IMOEX  2012-12-05  Индекс МосБиржи  Индекс МосБиржи  1435.97  1411.84  1441.45  1411.83  3.623375e+10       NaN    NaN         2    3.966432e+12        RUB      NaN              3   None
3    SNDX  IMOEX  2012-12-06  Индекс МосБиржи  Индекс МосБиржи  1438.54  1435.96  1449.44  1434.35  2.892222e+10       NaN    NaN         2    3.973540e+12        RUB      NaN              3   None
4    SNDX 

Нам интересны столбцы "TRADEDATE", "CLOSE", "OPEN", "HIGH", "LOW", "VALUE", "CAPITALIZATION". Также установим название индекса для каждого из них в названии столбцов, так как далее мы будем их гоизонтально объединять по "TRADEDATE" и анализировать переменные по каждому индексу.

In [ ]:
Subset = ['TRADEDATE','CLOSE','OPEN','HIGH','LOW','VALUE','CAPITALIZATION']
IMOEX_df = df[Subset]
Headings_IMOEX = {
    'TRADEDATE': 'TRADEDATE',
    'CLOSE': 'CLOSE_IMOEX',
    'OPEN': 'OPEN_IMOEX',
    'HIGH': 'HIGH_IMOEX',
    'LOW': 'LOW_IMOEX',
    'VALUE': 'VALUE_IMOEX',
    'CAPITALIZATION': 'CAPITALIZATION_IMOEX'
}
IMOEX_df = IMOEX_df.rename(columns = Headings_IMOEX)
print(IMOEX_df.head().to_string())
IMOEX_df.shape

    TRADEDATE  CLOSE_IMOEX  OPEN_IMOEX  HIGH_IMOEX  LOW_IMOEX   VALUE_IMOEX  CAPITALIZATION_IMOEX
0  2012-12-03      3177.05     3133.27     3178.49    3133.27  1.028820e+10          1.248851e+12
1  2012-12-04      3156.63     3177.05     3177.05    3145.32  9.829918e+09          1.240824e+12
2  2012-12-05      3206.90     3156.83     3223.56    3156.83  1.351435e+10          1.260583e+12
3  2012-12-06      3206.27     3206.90     3231.97    3199.36  1.159931e+10          1.260334e+12
4  2012-12-07      3216.69     3206.32     3221.52    3183.12  8.969189e+09          1.264428e+12


(2792, 7)

Эту же функцию реализуем для индекса металлов и добычи - MOEXMM; для индекса нефти и газа - MOEXOG; для индекса финансового сектора - IMOEXFN

In [ ]:
# для индекса MOEXMM

def get_moexmm(start_date, end_date, page_size = 100):
    moexmm_data = []
    count = 0
    while True:
        url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/MOEXMM.json?from={start_date}&till={end_date}&start={count}&limit={page_size}'
        try:
            response = requests.get(url, timeout=15)
            data = response.json()
            history_data = data['history']['data']
            headings = data['history']['columns']
            if history_data != None: 
                moexmm_data.extend(history_data)
                count += len(history_data)
        except:
            return pd.DataFrame(moexmm_data, columns=headings)
            
        if len(history_data) < page_size:
            break
    return pd.DataFrame(moexmm_data, columns=headings)

start_date = '2012-12-01'
end_date = '2024-01-31'
page_size = 100
df = get_moexmm(start_date, end_date, page_size=page_size)

In [33]:
Subset = ['TRADEDATE','CLOSE','OPEN','HIGH','LOW','VALUE','CAPITALIZATION']
MOEXMM_df = df[Subset]
Headings_MOEXMM = {
    'TRADEDATE': 'TRADEDATE',
    'CLOSE': 'CLOSE_MOEXMM',
    'OPEN': 'OPEN_MOEXMM',
    'HIGH': 'HIGH_MOEXMM',
    'LOW': 'LOW_MOEXMM',
    'VALUE': 'VALUE_MOEXMM',
    'CAPITALIZATION': 'CAPITALIZATION_MOEXMM'
}
MOEXMM_df = MOEXMM_df.rename(columns = Headings_MOEXMM)
print(MOEXMM_df.head().to_string())
MOEXMM_df.shape

    TRADEDATE  CLOSE_MOEXMM  OPEN_MOEXMM  HIGH_MOEXMM  LOW_MOEXMM  VALUE_MOEXMM  CAPITALIZATION_MOEXMM
0  2012-12-03       2760.50      2767.81      2767.90     2748.47  2.134668e+09           9.705972e+10
1  2012-12-04       2745.18      2760.35      2766.92     2735.92  3.232587e+09           9.652113e+10
2  2012-12-05       2809.80      2745.60      2812.41     2745.60  7.837108e+09           9.879316e+10
3  2012-12-06       2813.20      2809.80      2841.52     2806.10  6.283605e+09           9.891274e+10
4  2012-12-07       2813.67      2813.20      2820.26     2790.27  2.802574e+09           9.892925e+10


(2792, 7)

In [ ]:
# для индекса MOEXOG

def get_moexog(start_date, end_date, page_size = 100):
    moexog_data = []
    count = 0
    while True:
        url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/MOEXOG.json?from={start_date}&till={end_date}&start={count}&limit={page_size}'
        try:
            response = requests.get(url, timeout=15)
            data = response.json()
            history_data = data['history']['data']
            headings = data['history']['columns']
            if history_data != None: 
                moexog_data.extend(history_data)
                count += len(history_data)
        except:
            return pd.DataFrame(moexog_data, columns=headings)
            
        if len(history_data) < page_size:
            break
    return pd.DataFrame(moexog_data, columns=headings)

start_date = '2012-12-01'
end_date = '2024-01-31'
page_size = 100
df = get_moexog(start_date, end_date, page_size=page_size)

In [35]:
Subset = ['TRADEDATE','CLOSE','OPEN','HIGH','LOW','VALUE','CAPITALIZATION']
MOEXOG_df = df[Subset]
Headings_MOEXOG = {
    'TRADEDATE': 'TRADEDATE',
    'CLOSE': 'CLOSE_MOEXOG',
    'OPEN': 'OPEN_MOEXOG',
    'HIGH': 'HIGH_MOEXOG',
    'LOW': 'LOW_MOEXOG',
    'VALUE': 'VALUE_MOEXOG',
    'CAPITALIZATION': 'CAPITALIZATION_MOEXOG'
}
MOEXOG_df = MOEXOG_df.rename(columns = Headings_MOEXOG)
print(MOEXOG_df.head().to_string())
MOEXOG_df.shape

    TRADEDATE  CLOSE_MOEXOG  OPEN_MOEXOG  HIGH_MOEXOG  LOW_MOEXOG  VALUE_MOEXOG  CAPITALIZATION_MOEXOG
0  2012-12-03       3177.05      3133.27      3178.49     3133.27  1.028820e+10           1.248851e+12
1  2012-12-04       3156.63      3177.05      3177.05     3145.32  9.829918e+09           1.240824e+12
2  2012-12-05       3206.90      3156.83      3223.56     3156.83  1.351435e+10           1.260583e+12
3  2012-12-06       3206.27      3206.90      3231.97     3199.36  1.159931e+10           1.260334e+12
4  2012-12-07       3216.69      3206.32      3221.52     3183.12  8.969189e+09           1.264428e+12


(2792, 7)

In [55]:
# для индекса MOEXFN

def get_moexfn(start_date, end_date, page_size = 100):
    moexfn_data = []
    count = 0
    while True:
        url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/MOEXFN.json?from={start_date}&till={end_date}&start={count}&limit={page_size}'
        try:
            response = requests.get(url, timeout=15)
            data = response.json()
            history_data = data['history']['data']
            headings = data['history']['columns']
            if history_data != None: 
                moexfn_data.extend(history_data)
                count += len(history_data)
        except:
            return pd.DataFrame(moexfn_data, columns=headings)
            
        if len(history_data) < page_size:
            break
    return pd.DataFrame(moexfn_data, columns=headings)

start_date = '2012-12-01'
end_date = '2024-01-31'
page_size = 100
df = get_moexfn(start_date, end_date, page_size=page_size)

In [57]:
Subset = ['TRADEDATE','CLOSE','OPEN','HIGH','LOW','VALUE','CAPITALIZATION']
MOEXFN_df = df[Subset]
Headings_MOEXFN = {
    'TRADEDATE': 'TRADEDATE',
    'CLOSE': 'CLOSE_MOEX_FN',
    'OPEN': 'OPEN_MOEX_FN',
    'HIGH': 'HIGH_MOEX_FN',
    'LOW': 'LOW_MOEX_FN',
    'VALUE': 'VALUE_MOEX_FN',
    'CAPITALIZATION': 'CAPITALIZATION_MOEXFN'
}
MOEXFN_df = MOEXFN_df.rename(columns = Headings_MOEXFN)
print(MOEXFN_df.head().to_string())
MOEXFN_df.shape

    TRADEDATE  CLOSE_MOEX_FN  OPEN_MOEX_FN  HIGH_MOEX_FN  LOW_MOEX_FN  VALUE_MOEX_FN  CAPITALIZATION_MOEXFN
0  2012-12-03        4456.01       4427.65       4461.15      4420.24   1.052653e+10           3.877619e+10
1  2012-12-04        4408.71       4455.42       4455.42      4400.34   7.547101e+09           3.836455e+10
2  2012-12-05        4490.58       4408.71       4545.08      4408.71   1.143838e+10           3.907701e+10
3  2012-12-06        4486.50       4490.34       4541.58      4469.12   8.209059e+09           3.904148e+10
4  2012-12-07        4466.39       4486.74       4492.33      4440.74   9.837067e+09           3.886651e+10


(2792, 7)

### Объединение полученных датафреймов ###

In [58]:
result_df = pd.merge(IMOEX_df, MOEXMM_df, on='TRADEDATE', how='outer')
result_df = pd.merge(result_df, MOEXOG_df, on='TRADEDATE', how='outer')
result_df = pd.merge(result_df, MOEXFN_df, on='TRADEDATE', how='outer')
print(result_df.head().to_string())

    TRADEDATE  CLOSE_IMOEX  OPEN_IMOEX  HIGH_IMOEX  LOW_IMOEX   VALUE_IMOEX  CAPITALIZATION_IMOEX  CLOSE_MOEXMM  OPEN_MOEXMM  HIGH_MOEXMM  LOW_MOEXMM  VALUE_MOEXMM  CAPITALIZATION_MOEXMM  CLOSE_MOEXOG  OPEN_MOEXOG  HIGH_MOEXOG  LOW_MOEXOG  VALUE_MOEXOG  CAPITALIZATION_MOEXOG  CLOSE_MOEX_FN  OPEN_MOEX_FN  HIGH_MOEX_FN  LOW_MOEX_FN  VALUE_MOEX_FN  CAPITALIZATION_MOEXFN
0  2012-12-03      3177.05     3133.27     3178.49    3133.27  1.028820e+10          1.248851e+12       2760.50      2767.81      2767.90     2748.47  2.134668e+09           9.705972e+10       3177.05      3133.27      3178.49     3133.27  1.028820e+10           1.248851e+12        4456.01       4427.65       4461.15      4420.24   1.052653e+10           3.877619e+10
1  2012-12-04      3156.63     3177.05     3177.05    3145.32  9.829918e+09          1.240824e+12       2745.18      2760.35      2766.92     2735.92  3.232587e+09           9.652113e+10       3156.63      3177.05      3177.05     3145.32  9.829918e+09        

### Парсинг цен на аффинированные драгоценные металлы из данных Банка России ###

In [ ]:
# Для ЦБ мы уже не будем использовать API и JSON, поэтому нам надо воспользоваться библиотекой BeautifulSoup

start_date = '12.01.2012'
end_date = '31.01.2024'
# получаем доступ к сайту
url = f'https://cbr.ru/hd_base/metall/metall_base_new/?UniDbQuery.Posted=True&UniDbQuery.From=01.02.2012&UniDbQuery.To=27.02.2025&UniDbQuery.Gold=true&UniDbQuery.so=1'
response = requests.get(url) # проверяем есть ли у нас доступ
print(response)

<Response [200]>


In [107]:
# получаем HTML код страницы
tree = BeautifulSoup(response.text, 'html.parser')
table = tree.find('table', {'class': 'data'}) # если поисследовать код таблицы на сайте, то можно найти что она расположена в теге 'table', классе 'data'

data = []
for row in table.find_all('tr')[1:]: # не считываем заголовки,  'tr' в таблице обозначены строки
    cells = row.find_all('td') # td - столбцы
    date = pd.to_datetime(cells[0].text, dayfirst=True)
    price = cells[1].text.strip()
    price = price.replace(' ', '')
    price = price.replace(',', '') #  в данных ЦБ РФ были даны данные в формате '# ###,###', что помешало бы потом переводить их в числа, поэтому пришлось убирать знаки
    price = pd.to_numeric(price)
    data.append([date, price])

df = pd.DataFrame(data, columns=['Date', 'Price'])
print(df)

           Date     Price
0    2025-02-27  81035300
1    2025-02-26  81661500
2    2025-02-25  83209600
3    2025-02-22  83113400
4    2025-02-21  83573100
...         ...       ...
3225 2012-02-07  16689100
3226 2012-02-04  17105700
3227 2012-02-03  16959200
3228 2012-02-02  17049300
3229 2012-02-01  16938300

[3230 rows x 2 columns]
